In [1]:
import requests
from bs4 import BeautifulSoup as bs

import re
import unicodedata

import time
import random
import json

proxies = {
    'http': 'http://206.189.157.23',
    'http': 'http://206.189.157.23'
}

# Change the current working directory to repo 1st level
os.chdir('../')

In [2]:
# get links to hits for addiction recovery
base_url = 'https://www.churchofjesuschrist.org/search?lang=eng&query=addiction%20recovery'

In [4]:
def get_page(base_url, page, facet):
    get_payload = {
        'highlight':'true',
        'page': page,
        'facet':facet
    }
    r_get = requests.get(base_url, params=get_payload, 
                         proxies=proxies)
    return r_get
facets = ['general-conference', 'magazines']
# facets = ['general-conference', 'magazines', 'manuals']

page = 1
r_get = get_page(base_url, page=page, facet=facets[0])
print(r_get.url)
print("Status code: ", r_get.status_code)
print("Encoding: ", r_get.encoding)

# get urls : (page starts at 1, index, url)
soup = bs(r_get.text)
a_tags = soup.find_all('a', {'class' : 'sc-fodVek kfdvrS'})

urls = []
index = 0
for a_tag in a_tags:
    url = a_tag.get('href')
    urls.append((page, index, url))
    index += 1
urls[0:2]

https://www.churchofjesuschrist.org/search?lang=eng&query=addiction%20recovery&highlight=true&page=1&facet=general-conference
Status code:  200
Encoding:  utf-8


[(1,
  0,
  'https://www.churchofjesuschrist.org/study/general-conference/2009/10/repent-that-i-may-heal-you?lang=eng'),
 (1,
  1,
  'https://www.churchofjesuschrist.org/study/general-conference/2008/04/to-heal-the-shattering-consequences-of-abuse?lang=eng')]

## URL Crawler

In [30]:
!python crawler.py \
--base_url "https://www.churchofjesuschrist.org/search?lang=eng&query=addiction%20recovery" \
--facet general-conference\
--start_page 1\
--end_page 1000\
--last_index 0\
--save_file "data/general-conference-1-1000.json"
# use magazines-1-100.json for magazines facet

Crawling general-conference pages 1 to 1000...
100%|███████████████████████████████████████| 1000/1000 [13:29<00:00,  1.24it/s]
Done.


In [32]:
!python crawler.py \
--base_url "https://www.churchofjesuschrist.org/search?lang=eng&query=addiction%20recovery" \
--facet magazines\
--start_page 1\
--end_page 1000\
--last_index 0\
--save_file "data/magazines-1-1000.json"

Crawling magazines pages 1 to 1000...
100%|███████████████████████████████████████| 1000/1000 [16:53<00:00,  1.01s/it]
Done.


In [5]:
# read the file
filename = "data/general-conference-1-1000.json"
with open(filename) as f:
    urls_list = [tuple(x) for x in json.load(f)]
urls_list[0:2]

[(1,
  0,
  'https://www.churchofjesuschrist.org/study/general-conference/2009/10/repent-that-i-may-heal-you?lang=eng'),
 (1,
  1,
  'https://www.churchofjesuschrist.org/study/general-conference/2008/04/to-heal-the-shattering-consequences-of-abuse?lang=eng')]

## Scraper

In [6]:
r_get = requests.get('https://www.churchofjesuschrist.org/study/general-conference/2014/04/are-you-sleeping-through-the-restoration')

# get urls : (page starts at 1, index, url)
soup = bs(r_get.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <title data-react-helmet="true">
   Are You Sleeping through the Restoration?
  </title>
  <meta content="3.4.0" name="version"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <meta content="28860731" data-react-helmet="true" name="Search.doc-aid"/>
  <meta content="Are You Sleeping through the Restoration?" data-react-helmet="true" name="title"/>
  <meta content="Dieter F. Uchtdorf cautions that selfishness, addictions, and competing priorities can keep us from giving our best effort as disciples of Christ." data-react-helmet="true" name="description"/>
  <meta content="https://assets.ldscdn.org/7f/55/7f559f23d9b7403ce31c195c8d47c377d5af4b85/7f559f23d9b7403ce31c195c8d47c377d5af4b85.jpeg" data-react-helmet="true" property="og:image"/>
  <meta content="Are You Sleeping through the Restoration?" data-react-helmet="true" property="og:title"/>
  <meta content="website" data-react-helmet="true" property="o

In [7]:
def clean_text(text):
     # fix formatting, decode without the BOM (byte order mark)
    text = text.encode('iso-8859-1').decode('utf-8-sig')
    text = unicodedata.normalize('NFKD', text)
    return text

In [8]:
# title
title = clean_text(soup.head.title.text)
print(title)

kicker = clean_text(soup.select_one('p#kicker1.kicker').text)
print(kicker)

Are You Sleeping through the Restoration?
There is too much at stake for us as individuals, as families, and as Christ’s Church to give only a halfhearted effort to this sacred work.


In [10]:
# body
body_element = soup.find('div', {"class":'body-block'})

body_text = []
for p in body_element.find_all('p'):
    # fix formatting, decode withou the BOM (byte order mark)
    paragraph = clean_text(p.text)
    body_text.append(paragraph)

body = "\n".join(body_text)
body[0:200]

'Nearly 200 years ago, the American short story “Rip Van Winkle” became an instant classic. The main character, Rip, is an unambitious man who is very good at avoiding two things: work and his wife.\nOn'

In [11]:
# author-calling
bylines = soup.find_all('div', {'class' : 'byline'})
author = clean_text(bylines[0].find(id='p1').text)
author = re.sub('By ', '', author)
print(author)

calling = clean_text(bylines[0].find(id='p2').text)
print(calling)

President Dieter F. Uchtdorf
Second Counselor in the First Presidency


### Scrape `general-conference`

In [29]:
!python scraper.py \
--source_file "data/general-conference-1-1000.json"\
--save_dir "data/general-conference"

Scraping general-conference pages...
 50%|█████████████████▍                 | 4476/8965 [8:00:14<8:35:48,  6.89s/it]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



100%|████████████████████████████████████| 8965/8965 [15:53:03<00:00,  6.38s/it]
Done.


In [ ]:
file.split('.')[1].split('/')[-1].split('-')[-1]

In [51]:
# missing data report (include this step in scraper)
from glob import glob
from tqdm import tqdm

def inspect_missing(glob_string):
    scraped_files = glob(glob_string)

    missing_report = {
        'title': [],
        'kicker': [],
        'body': [],
        'author': [],
        'calling': []
    }
    for file in tqdm(scraped_files):
        index = int(file.split('.')[1].split('/')[-1].split('-')[-1])

        # read the file
        with open(file) as json_file:
            data = json.load(json_file)

        if not data['title']:
            missing_report['title'].append(index)

        if not data['kicker']:
            missing_report['kicker'].append(index)

        if not data['body']:
            missing_report['body'].append(index)

        if not data['author']:
            missing_report['author'].append(index)

        if not data['calling']:
            missing_report['calling'].append(index)
    return missing_report

gc_missing_report = inspect_missing('./general-conference/*.json')

100%|██████████| 8965/8965 [00:04<00:00, 2049.53it/s]


In [42]:
# 997 files do not have body
len(gc_missing_report['body'])

997

In [49]:
# inspect : all 997 objects refer to the same landing page 
# of October 1981 conference talk list
filename = "data/general-conference-1-1000.json"
with open(filename) as f:
    urls_list = [tuple(x) for x in json.load(f)]

for index in gc_missing_report['body']:
    print(urls_list[index][-1])

https://www.churchofjesuschrist.org/study/general-conference/1981/10/_manifest?lang=eng
https://www.churchofjesuschrist.org/study/general-conference/1981/10/_manifest?lang=eng
https://www.churchofjesuschrist.org/study/general-conference/1981/10/_manifest?lang=eng
https://www.churchofjesuschrist.org/study/general-conference/1981/10/_manifest?lang=eng
https://www.churchofjesuschrist.org/study/general-conference/1981/10/_manifest?lang=eng
https://www.churchofjesuschrist.org/study/general-conference/1981/10/_manifest?lang=eng
https://www.churchofjesuschrist.org/study/general-conference/1981/10/_manifest?lang=eng
https://www.churchofjesuschrist.org/study/general-conference/1981/10/_manifest?lang=eng
https://www.churchofjesuschrist.org/study/general-conference/1981/10/_manifest?lang=eng
https://www.churchofjesuschrist.org/study/general-conference/1981/10/_manifest?lang=eng
https://www.churchofjesuschrist.org/study/general-conference/1981/10/_manifest?lang=eng
https://www.churchofjesuschrist.

In [ ]:
# !python scraper.py \
# --source_file "data/general-conference-1-1000.json"\
# --save_dir "data/general-conference"
# --start_index
# --end_index
# --select_index

### Scrape `magazines`

In [52]:
!python scraper.py \
--source_file "data/magazines-1-1000.json"\
--save_dir "data/magazines"

Scraping magazines pages...
 69%|███████████████████████▌          | 6930/9978 [10:48:30<4:38:14,  5.48s/it]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



100%|████████████████████████████████████| 9978/9978 [15:24:01<00:00,  5.56s/it]
Done.


In [53]:
# inspect using missing report file
# load report file
report_file = 'data/magazines-missing-report.json'
with open(report_file) as f:
    mag_missing_report = json.load(f)

filename = "data/general-conference-1-1000.json"
with open(filename) as f:
    urls_list = [tuple(x) for x in json.load(f)]

for index in mag_missing_report['body']:
    print(urls_list[index][-1])

https://www.churchofjesuschrist.org/study/general-conference/2010/10/o-that-cunning-plan-of-the-evil-one?lang=eng
https://www.churchofjesuschrist.org/study/general-conference/2014/04/protection-from-pornography-a-christ-focused-home?lang=eng
https://www.churchofjesuschrist.org/study/general-conference/1981/10?lang=eng
https://www.churchofjesuschrist.org/study/general-conference/2014/04/are-you-sleeping-through-the-restoration
https://www.churchofjesuschrist.org/study/general-conference/2021/04/34eyring?lang=eng
https://www.churchofjesuschrist.org/study/general-conference/2007/10/blessed-are-all-the-pure-in-heart?lang=ase
https://www.churchofjesuschrist.org/study/general-conference/2009/10/repent-that-i-may-heal-you?lang=eng
https://www.churchofjesuschrist.org/study/general-conference/2014/04/are-you-sleeping-through-the-restoration
https://www.churchofjesuschrist.org/study/general-conference/1988/10/addiction-or-freedom?lang=eng
https://www.churchofjesuschrist.org/study/general-confere

In [54]:
len(mag_missing_report['body'])

17